In [1]:
import zipfile
import os

# ZIP file ka path
zip_path = r"C:\Users\Admin\Desktop\Mircosoft\projects\pages\archive (3).zip"

# Jahan extract karna hai
extract_path = r"C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset"

# Extract ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check kya extract hua
print("Extracted Files:", os.listdir(extract_path))

Extracted Files: ['classified_dataset', 'Comprehensive Disaster Dataset(CDD)', 'split_dataset']


In [2]:
import os

# Path to CDD folder
cdd_path = r"C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\Comprehensive Disaster Dataset(CDD)"

# Dekho andhar kya kya hai
for root, dirs, files in os.walk(cdd_path):
    print(f"\n📁 Folder: {root}")
    print(f"   📂 Subfolders: {dirs}")
    print(f"   📄 Files: {files[:5]}")  # sirf 5 files dikha rahe for preview



📁 Folder: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\Comprehensive Disaster Dataset(CDD)
   📂 Subfolders: ['Damaged_Infrastructure', 'Fire_Disaster', 'Human_Damage', 'Land_Disaster', 'Water_Disaster']
   📄 Files: []

📁 Folder: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\Comprehensive Disaster Dataset(CDD)\Damaged_Infrastructure
   📂 Subfolders: ['Earthquake', 'Infrastructure']
   📄 Files: []

📁 Folder: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\Comprehensive Disaster Dataset(CDD)\Damaged_Infrastructure\Earthquake
   📂 Subfolders: []
   📄 Files: ['05_02_0001.png', '05_02_0002.png', '05_02_0003.png', '05_02_0004.png', '05_02_0005.png']

📁 Folder: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\Comprehensive Disaster Dataset(CDD)\Damaged_Infrastructure\Infrastructure
   📂 Subfolders: []
   📄 Files: ['05_01_0001.png', '05_01_0002.png', '05_01_0003.png', '05_01_0004.png', '05_01_0005.png']

📁 Folder: C:\Users

In [23]:
for root, dirs, files in os.walk(cdd_path):
    print(f"\n📁 Folder: {root}")
    print(f"   📂 Subfolders: {dirs}")
    print(f"   📄 Files: {files[:5]}")  # sirf 5 files dikha rahe for preview



📁 Folder: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\Comprehensive Disaster Dataset(CDD)
   📂 Subfolders: ['Damaged_Infrastructure', 'Fire_Disaster', 'Human_Damage', 'Land_Disaster', 'Water_Disaster']
   📄 Files: []

📁 Folder: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\Comprehensive Disaster Dataset(CDD)\Damaged_Infrastructure
   📂 Subfolders: ['Earthquake', 'Infrastructure']
   📄 Files: []

📁 Folder: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\Comprehensive Disaster Dataset(CDD)\Damaged_Infrastructure\Earthquake
   📂 Subfolders: []
   📄 Files: ['05_02_0001.png', '05_02_0002.png', '05_02_0003.png', '05_02_0004.png', '05_02_0005.png']

📁 Folder: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\Comprehensive Disaster Dataset(CDD)\Damaged_Infrastructure\Infrastructure
   📂 Subfolders: []
   📄 Files: ['05_01_0001.png', '05_01_0002.png', '05_01_0003.png', '05_01_0004.png', '05_01_0005.png']

📁 Folder: C:\Users

In [3]:
import os
import shutil
import random

original_base = r"C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\Comprehensive Disaster Dataset(CDD)"
output_base = r"C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\classified_dataset"

# Folder mapping (you can adjust these)
category_map = {
    "Fire_Disaster/Urban_Fire": "urban_fire",
    "Fire_Disaster/Wild_Fire": "wild_fire",
    "Land_Disaster/Land_Slide": "landslide",
    "Land_Disaster/Drought": "drought",
    "Water_Disaster": "flood",
    "Damaged_Infrastructure/Earthquake": "earthquake",
    "Damaged_Infrastructure/Infrastructure": "infra_damage",
    "Human_Damage": "casualties"
}

def split_and_copy():
    for src_rel, label in category_map.items():
        src_folder = os.path.join(original_base, src_rel)
        images = [f for f in os.listdir(src_folder) if f.endswith(".png")]
        random.shuffle(images)

        split_idx = int(0.8 * len(images))
        train_imgs = images[:split_idx]
        val_imgs = images[split_idx:]

        for phase, files in [("train", train_imgs), ("val", val_imgs)]:
            dest_folder = os.path.join(output_base, phase, label)
            os.makedirs(dest_folder, exist_ok=True)
            for f in files:
                shutil.copy(os.path.join(src_folder, f), os.path.join(dest_folder, f))

    print("✅ Dataset organized for training.")

split_and_copy()


✅ Dataset organized for training.


In [4]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image, UnidentifiedImageError
from torchvision.models import resnet18, ResNet18_Weights

# ✅ Use CPU if GPU not available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Using device: {device}")

# ✅ Handle corrupt images safely
class SafeImageFolder(ImageFolder):
    def __getitem__(self, index):
        try:
            return super(SafeImageFolder, self).__getitem__(index)
        except (UnidentifiedImageError, OSError):
            print(f"⚠️ Skipping corrupt image: {self.samples[index][0]}")
            new_index = (index + 1) % len(self.samples)
            return self.__getitem__(new_index)

# ✅ Paths
data_dir = r"C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\classified_dataset"

# ✅ Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# ✅ Load datasets safely
train_data = SafeImageFolder(os.path.join(data_dir, "train"), transform=transform)
val_data = SafeImageFolder(os.path.join(data_dir, "val"), transform=transform)

# ✅ Class names
class_names = train_data.classes
print("Classes:", class_names)

# ✅ DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)

# ✅ Model - ResNet18
model = resnet18(weights=ResNet18_Weights.DEFAULT)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(class_names))  # Output layer as per our classes
model = model.to(device)

# ✅ Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"🧪 Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}, Accuracy: {acc:.2f}%")

# ✅ Save the model
torch.save(model.state_dict(), "disaster_classifier.pth")
print("✅ Model saved as disaster_classifier.pth")

🖥️ Using device: cpu
Classes: ['casualties', 'drought', 'earthquake', 'flood', 'infra_damage', 'landslide', 'urban_fire', 'wild_fire']
⚠️ Skipping corrupt image: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\classified_dataset\train\infra_damage\05_01_0261.png
⚠️ Skipping corrupt image: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\classified_dataset\train\casualties\02_0069.png
🧪 Epoch [1/5], Loss: 121.0743, Accuracy: 69.85%
⚠️ Skipping corrupt image: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\classified_dataset\train\infra_damage\05_01_0261.png
⚠️ Skipping corrupt image: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\classified_dataset\train\casualties\02_0069.png
🧪 Epoch [2/5], Loss: 86.6954, Accuracy: 77.53%
⚠️ Skipping corrupt image: C:\Users\Admin\Desktop\Mircosoft\projects\pages\disaster_dataset\classified_dataset\train\infra_damage\05_01_0261.png
⚠️ Skipping corrupt image: C:\Users\Admin\Desktop\Mirco

In [5]:
import pickle

# Save the trained model using pickle
with open("disasters_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ Model successfully saved as disaster_model.pkl")


✅ Model successfully saved as disaster_model.pkl
